In [1]:
from pandas.io.json import json_normalize
from sodapy import Socrata
from elasticsearch import Elasticsearch

import json
import requests
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta 
import os, time
import pytz
import arrow

In [ ]:
# get credentials
with open('.credentials') as f:
    credentials = f.readlines()
    credentials = json.loads(''.join(credentials).strip())

In [2]:
client = Socrata('www.data.act.gov.au',
                 '4jqogoRJ9NKj1gr8QAZ9CCKFI',
                 username=credentials['username'],
                 password=credentials['password'])

In [3]:
results = client.get("jxpp-4iiz", limit=19000)

In [4]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

## Converting, Sorting and Dropping Columns

In [5]:
results_df['arrival_time'] = pd.to_datetime(results_df['arrival_time'])
results_df['depature_time'] = pd.to_datetime(results_df['depature_time'])
results_df['timestamp'] = pd.to_datetime(results_df['timestamp'])

In [6]:
results_df = results_df.sort_values(by='arrival_time',ascending=True)

In [7]:
results_df.head()

,arrival_delay,arrival_time,arrivaluncertainty,depature_delay,depature_time,depatureuncertainty,gtfsrupdateid,stop_id,stop_sequence,timestamp,trip_id,tripschedulerelationship,tripupdateschedulerelationship,updatedeleted
8158,127,2019-06-19 06:13:07,0,127,2019-06-19 06:13:07,0,2496736,8100,5,2019-06-19 23:59:45,4,SCHEDULED,SCHEDULED,None
8220,149,2019-06-19 06:14:58,0,149,2019-06-19 06:14:58,0,2496798,8129,8,2019-06-19 23:59:45,134,SCHEDULED,SCHEDULED,None
8118,-4,2019-06-19 06:23:56,0,-4,2019-06-19 06:23:56,0,2496699,8129,13,2019-06-19 23:59:45,135,SCHEDULED,SCHEDULED,None
8184,194,2019-06-19 06:27:14,0,194,2019-06-19 06:27:14,0,2496762,8100,13,2019-06-19 23:59:45,5,SCHEDULED,SCHEDULED,None
8159,6,2019-06-19 06:39:06,0,6,2019-06-19 06:39:06,0,2496737,8129,13,2019-06-19 23:59:45,136,SCHEDULED,SCHEDULED,None


In [8]:
results_df.tail()

,arrival_delay,arrival_time,arrivaluncertainty,depature_delay,depature_time,depatureuncertainty,gtfsrupdateid,stop_id,stop_sequence,timestamp,trip_id,tripschedulerelationship,tripupdateschedulerelationship,updatedeleted
18975,12,2019-06-26 10:26:56,0,12,2019-06-26 10:27:16,0,4891830,8121,5,2019-06-26 10:27:15,49,SCHEDULED,SCHEDULED,None
18955,-28,2019-06-26 10:27:03,0,-28,2019-06-26 10:27:23,0,4891689,8110,5,2019-06-26 10:27:15,173,SCHEDULED,SCHEDULED,None
13372,-19,2019-06-26 10:31:43,0,-19,2019-06-26 10:32:03,0,4895697,8105,12,2019-06-26 10:31:15,48,SCHEDULED,SCHEDULED,None
15218,-13,2019-06-26 10:32:00,0,-13,2019-06-26 10:32:20,0,4895652,8126,12,2019-06-26 10:31:15,172,SCHEDULED,SCHEDULED,None
12248,0,2019-06-26 11:07:31,1,0,2019-06-26 11:07:51,1,4895548,8110,5,2019-06-26 10:31:15,177,SCHEDULED,SCHEDULED,None


In [9]:
results_df = results_df[['arrival_delay', 'arrival_time', 'arrivaluncertainty', 'depature_delay', 'depature_time', 'depatureuncertainty', 'stop_id', 'stop_sequence', 'trip_id']]

In [10]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19000 entries, 8158 to 12248
Data columns (total 9 columns):
arrival_delay          19000 non-null object
arrival_time           19000 non-null datetime64[ns]
arrivaluncertainty     19000 non-null object
depature_delay         19000 non-null object
depature_time          19000 non-null datetime64[ns]
depatureuncertainty    19000 non-null object
stop_id                19000 non-null object
stop_sequence          19000 non-null object
trip_id                19000 non-null object
dtypes: datetime64[ns](2), object(7)
memory usage: 1.4+ MB


In [11]:
results_df['arrival_delay'] = results_df['arrival_delay'].astype(int)
results_df['depature_delay'] = results_df['depature_delay'].astype(int)
results_df['stop_sequence'] = results_df['stop_sequence'].astype(int)

In [12]:
results_df.head()

,arrival_delay,arrival_time,arrivaluncertainty,depature_delay,depature_time,depatureuncertainty,stop_id,stop_sequence,trip_id
8158,127,2019-06-19 06:13:07,0,127,2019-06-19 06:13:07,0,8100,5,4
8220,149,2019-06-19 06:14:58,0,149,2019-06-19 06:14:58,0,8129,8,134
8118,-4,2019-06-19 06:23:56,0,-4,2019-06-19 06:23:56,0,8129,13,135
8184,194,2019-06-19 06:27:14,0,194,2019-06-19 06:27:14,0,8100,13,5
8159,6,2019-06-19 06:39:06,0,6,2019-06-19 06:39:06,0,8129,13,136


## Combine Tables

In [13]:
trips = pd.read_csv('./GTFS/google_transit_lr/trips.csv')
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 9 columns):
route_id                 847 non-null object
service_id               847 non-null object
trip_id                  847 non-null int64
trip_headsign            847 non-null object
direction_id             847 non-null int64
block_id                 847 non-null object
shape_id                 847 non-null int64
wheelchair_accessible    847 non-null int64
bikes_allowed            847 non-null int64
dtypes: int64(5), object(4)
memory usage: 59.6+ KB


In [14]:
trips = trips[['route_id', 'service_id', 'trip_id','trip_headsign', 'direction_id']]

In [15]:
trips['trip_id'] = trips['trip_id'].astype(str)

In [16]:
df = results_df.set_index('arrival_time')

## Function

In [17]:
def getDate(date, df):
    df_copy = df.copy(deep=True)
    df_date = df_copy.loc[date]
    df_date.reset_index(inplace=True)
    
    trips = pd.read_csv('./GTFS/google_transit_lr/trips.csv')
    trips['trip_id'] = trips['trip_id'].astype(str)
    
    date = pd.merge(left=df_date, left_on='trip_id', right=trips, right_on='trip_id', how='left')
    date = date[date.columns[:-4]]
    date['arrival_time'] = date['arrival_time'].apply(lambda x: arrow.get(x, 'Australia/Canberra').datetime)
    date['depature_time'] = date['depature_time'].apply(lambda x: arrow.get(x, 'Australia/Canberra').datetime)
    
    print(date.columns)
    grouped = date.groupby('route_id')
    
    full_service = grouped.get_group('ACTO001')
    x1 = grouped.get_group('X1')
    try:
        x2 = grouped.get_group('X2')
    except:
        return [full_service, x1]
    
    return [full_service, x1, x2]

In [18]:
june_20 = getDate('2019-06-20', df)
june_21 = getDate('2019-06-21', df)
june_22 = getDate('2019-06-22', df)
june_23 = getDate('2019-06-23', df)
june_24 = getDate('2019-06-24', df)
june_25 = getDate('2019-06-25', df)

Index(['arrival_time', 'arrival_delay', 'arrivaluncertainty', 'depature_delay',
       'depature_time', 'depatureuncertainty', 'stop_id', 'stop_sequence',
       'trip_id', 'route_id', 'service_id', 'trip_headsign', 'direction_id'],
      dtype='object')
Index(['arrival_time', 'arrival_delay', 'arrivaluncertainty', 'depature_delay',
       'depature_time', 'depatureuncertainty', 'stop_id', 'stop_sequence',
       'trip_id', 'route_id', 'service_id', 'trip_headsign', 'direction_id'],
      dtype='object')
Index(['arrival_time', 'arrival_delay', 'arrivaluncertainty', 'depature_delay',
       'depature_time', 'depatureuncertainty', 'stop_id', 'stop_sequence',
       'trip_id', 'route_id', 'service_id', 'trip_headsign', 'direction_id'],
      dtype='object')
Index(['arrival_time', 'arrival_delay', 'arrivaluncertainty', 'depature_delay',
       'depature_time', 'depatureuncertainty', 'stop_id', 'stop_sequence',
       'trip_id', 'route_id', 'service_id', 'trip_headsign', 'direction_id'],
  

In [54]:
x = june_20[1]
y = x[x['trip_id'] == '266']
y

,arrival_time,arrival_delay,arrivaluncertainty,depature_delay,depature_time,depatureuncertainty,stop_id,stop_sequence,trip_id,route_id,service_id,trip_headsign,direction_id
0,2019-06-20 00:00:00+10:00,0,1,0,2019-06-20 00:00:00+10:00,1,8100,1,266,X1,WD,Well Station,1
1,2019-06-20 00:01:12+10:00,-172,0,-172,2019-06-20 00:01:32+10:00,0,8106,3,266,X1,WD,Well Station,1
2,2019-06-20 00:02:48+10:00,-172,0,-172,2019-06-20 00:03:08+10:00,0,8108,4,266,X1,WD,Well Station,1
3,2019-06-20 00:04:39+10:00,-172,0,-172,2019-06-20 00:04:39+10:00,0,8110,5,266,X1,WD,Well Station,1


## Elasticsearch

In [47]:
es=Elasticsearch([{'host':'localhost','port':9200, 'http_auth':('elastic', 'changeme')}])

In [48]:
counter = 1

In [49]:
for record in june_20:
    for idx, value in enumerate(record.to_dict(orient='records')):
        es.index(index='trips', doc_type='lightrail', body=value, id=counter)
        counter+=1

In [50]:
for record in june_21:
    for idx, value in enumerate(record.to_dict(orient='records')):
        es.index(index='trips', doc_type='lightrail', body=value, id=counter)
        counter+=1

In [51]:
for record in june_22:
    for idx, value in enumerate(record.to_dict(orient='records')):
        es.index(index='trips', doc_type='lightrail', body=value, id=counter)
        counter+=1

In [52]:
for record in june_23:
    for idx, value in enumerate(record.to_dict(orient='records')):
        es.index(index='trips', doc_type='lightrail', body=value, id=counter)
        counter+=1

In [53]:
for record in june_24:
    for idx, value in enumerate(record.to_dict(orient='records')):
        es.index(index='trips', doc_type='lightrail', body=value, id=counter)
        counter+=1

---

## Plotly

In [55]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [56]:
iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])

## June 20

In [60]:
full_service = june_20[0]

In [61]:
full_service['trip_id'].unique()

array(['5', '135', '6', '136', '137', '7', '140', '12', '141', '15',
       '142', '18', '143', '144', '21', '145', '22', '146', '147', '23',
       '24', '148', '149', '25', '150', '26', '27', '151', '152', '28',
       '153', '29', '154', '30', '155', '31', '32', '156', '33', '157',
       '34', '158', '35', '159', '36', '160', '161', '37', '38', '162',
       '165', '41', '44', '168', '169', '45', '170', '46', '171', '47',
       '48', '172', '173', '49', '174', '50', '175', '51', '52', '176',
       '53', '177', '54', '178', '55', '179', '180', '56', '57', '181',
       '58', '182', '183', '59', '60', '184', '185', '61', '62', '186',
       '187', '63', '188', '64', '65', '189', '190', '66', '67', '191',
       '68', '192', '69', '193', '70', '194', '71', '195', '72', '196',
       '197', '73', '74', '198', '75', '199', '76', '200', '77', '201',
       '78', '202', '79', '203', '204', '80', '207', '83', '86', '210',
       '87', '211', '88', '212', '89', '213', '214', '90', '215', 

In [63]:
trip_ = full_service[full_service['trip_id'] == '5']

In [67]:
trip_

,arrival_time,arrival_delay,arrivaluncertainty,depature_delay,depature_time,depatureuncertainty,stop_id,stop_sequence,trip_id,route_id,service_id,trip_headsign,direction_id
5,2019-06-20 06:00:00+10:00,0,1,0,2019-06-20 06:00:00+10:00,1,8129,1,5,ACTO001,WD,Gungahlin Pl,0
7,2019-06-20 06:01:02+10:00,-52,0,-52,2019-06-20 06:01:22+10:00,0,8127,2,5,ACTO001,WD,Gungahlin Pl,0
10,2019-06-20 06:02:36+10:00,-48,0,-48,2019-06-20 06:02:56+10:00,0,8125,3,5,ACTO001,WD,Gungahlin Pl,0
12,2019-06-20 06:03:56+10:00,-49,0,-49,2019-06-20 06:04:16+10:00,0,8123,4,5,ACTO001,WD,Gungahlin Pl,0
16,2019-06-20 06:06:01+10:00,-43,0,-43,2019-06-20 06:06:21+10:00,0,8121,5,5,ACTO001,WD,Gungahlin Pl,0
21,2019-06-20 06:08:34+10:00,-7,0,-7,2019-06-20 06:08:54+10:00,0,8119,6,5,ACTO001,WD,Gungahlin Pl,0
24,2019-06-20 06:10:05+10:00,-40,0,-40,2019-06-20 06:10:25+10:00,0,8117,7,5,ACTO001,WD,Gungahlin Pl,0
28,2019-06-20 06:12:01+10:00,-59,0,-59,2019-06-20 06:12:21+10:00,0,8115,8,5,ACTO001,WD,Gungahlin Pl,0
37,2019-06-20 06:16:03+10:00,-30,0,-30,2019-06-20 06:16:23+10:00,0,8111,9,5,ACTO001,WD,Gungahlin Pl,0
40,2019-06-20 06:18:30+10:00,-7,0,-7,2019-06-20 06:18:50+10:00,0,8109,10,5,ACTO001,WD,Gungahlin Pl,0


In [72]:
trace = go.Scatter(
    x = trip_['arrival_time'].apply(lambda x: x.replace(tzinfo=None)),
    y = trip_['stop_sequence']
)

data = [trace]
fig = go.Figure(data)
iplot(fig)

In [73]:
data = []
for trips in full_service['trip_id'].unique():
    trip_ = full_service[full_service['trip_id'] == trips]
    trace = go.Scatter(
    x = trip_['arrival_time'].apply(lambda x: x.replace(tzinfo=None)),
    y = trip_['stop_sequence'],
    name = trips)
    data.append(trace)
fig = go.Figure(data)
iplot(fig)

### To Gungahlin

In [77]:
to_gungahlin = full_service[full_service['trip_headsign'] == 'Gungahlin Pl']

In [78]:
data = []
for trips in to_gungahlin['trip_id'].unique():
    trip_ = to_gungahlin[to_gungahlin['trip_id'] == trips]
    trace = go.Scatter(
    x = trip_['arrival_time'].apply(lambda x: x.replace(tzinfo=None)),
    y = trip_['stop_sequence'],
    name = trips)
    data.append(trace)
fig = go.Figure(data)
iplot(fig)

In [81]:
to_gungahlin[to_gungahlin['trip_id'] == '33']

,arrival_time,arrival_delay,arrivaluncertainty,depature_delay,depature_time,depatureuncertainty,stop_id,stop_sequence,trip_id,route_id,service_id,trip_headsign,direction_id
527,2019-06-20 08:34:06+10:00,-228,0,-228,2019-06-20 08:34:26+10:00,0,8127,2,33,ACTO001,WD,Gungahlin Pl,0
534,2019-06-20 08:35:36+10:00,-228,0,-228,2019-06-20 08:35:56+10:00,0,8125,3,33,ACTO001,WD,Gungahlin Pl,0
537,2019-06-20 08:36:00+10:00,0,1,0,2019-06-20 08:36:00+10:00,1,8129,1,33,ACTO001,WD,Gungahlin Pl,0
542,2019-06-20 08:36:57+10:00,-228,0,-228,2019-06-20 08:37:17+10:00,0,8123,4,33,ACTO001,WD,Gungahlin Pl,0
551,2019-06-20 08:38:56+10:00,-228,0,-228,2019-06-20 08:39:16+10:00,0,8121,5,33,ACTO001,WD,Gungahlin Pl,0
560,2019-06-20 08:40:53+10:00,-228,0,-228,2019-06-20 08:41:13+10:00,0,8119,6,33,ACTO001,WD,Gungahlin Pl,0
569,2019-06-20 08:42:57+10:00,-228,0,-228,2019-06-20 08:43:17+10:00,0,8117,7,33,ACTO001,WD,Gungahlin Pl,0
579,2019-06-20 08:45:12+10:00,-228,0,-228,2019-06-20 08:45:32+10:00,0,8115,8,33,ACTO001,WD,Gungahlin Pl,0
596,2019-06-20 08:48:45+10:00,-228,0,-228,2019-06-20 08:49:05+10:00,0,8111,9,33,ACTO001,WD,Gungahlin Pl,0
607,2019-06-20 08:50:49+10:00,-228,0,-228,2019-06-20 08:51:09+10:00,0,8109,10,33,ACTO001,WD,Gungahlin Pl,0
